In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import feast
import shutil

from nvtabular import ColumnSchema, Schema
from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.session_filter import FilterCandidates
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures
from merlin.systems.triton.utils import send_triton_request

#### Define output path for data

In [ ]:
BASE_DIR = "/workdir"
FEATURE_STORE_DIR = os.path.join(BASE_DIR, "feature_repo/")
TRITON_MODEL_REPO = os.path.join(BASE_DIR, "models/")

DATA_DIR = "/model-data/"
DLRM_DIR = os.path.join(DATA_DIR, "dlrm")
QUERY_TOWER_DIR = os.path.join(DATA_DIR, "query_tower")
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")

#### Define Triton Ensemble

In [ ]:
retrieval_model_path = os.path.join(TRITON_MODEL_REPO, "1-user-embeddings/1/model.savedmodel/")
ranking_model_path = os.path.join(TRITON_MODEL_REPO, "5-ranking/1/model.savedmodel/")

# Copy over pretrined Query Tower Model to our Triton Model Repository
if not os.path.isdir(retrieval_model_path):
    shutil.copytree(QUERY_TOWER_DIR, retrieval_model_path)

# Copy over pretrined DLRMfor ranking to our Triton Model Repository
if not os.path.isdir(ranking_model_path):
    shutil.copytree(DLRM_DIR, ranking_model_path)

In [ ]:
retrieval_model_path = os.path.join(TRITON_MODEL_REPO, "1-user-embeddings/1/model.savedmodel/")
ranking_model_path = os.path.join(TRITON_MODEL_REPO, "5-ranking/1/model.savedmodel/")

# Copy over pretrined Query Tower Model to our Triton Model Repository
if not os.path.isdir(retrieval_model_path):
    shutil.copytree(QUERY_TOWER_DIR, retrieval_model_path)

# Copy over pretrined DLRMfor ranking to our Triton Model Repository
if not os.path.isdir(ranking_model_path):
    shutil.copytree(DLRM_DIR, ranking_model_path)

#### Start triton server

In [ ]:
!tritonserver --model-repository=/workdir/models --backend-config=tensorflow,version=2

#### Retrieving Recommendations from Triton

In [ ]:
!python client.py --user 23

In [ ]:
!perf_analyzer -m ensemble-model -u localhost:8000 --input-data=sample.json --shape=user_id_raw:1,1 -t 2